In [3]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from keras.datasets import mnist
from sklearn.linear_model import Perceptron

from sklearn.metrics import classification_report, accuracy_score, log_loss

from sklearn.decomposition import PCA

import numpy as np
import time

In [4]:
(xtrain,ytrain),(xtest,ytest) = mnist.load_data()
xtrain = xtrain.reshape(60000, 28*28).astype('float32')
xtest = xtest.reshape(10000, 28*28).astype('float32')

xtrain /= 255
xtest /= 255

xtrain2 = xtrain[:10000,:]
ytrain2 = ytrain[:10000]

11490434/11490434 [==============================] - 0s 0us/step


In [6]:
pca = PCA(n_components=100)
pca.fit(xtrain)
pca_xtrain = pca.transform(xtrain)
pca_xtest = pca.transform(xtest)

In [3]:
#Perceptron sans preprocessing
start_time = time.time()
pctr = Perceptron(tol=1e-3, random_state=0)
pctr.fit(xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = pctr.predict(xtest)

Temps entraînement = 8.896295547485352s


In [5]:
accuracy_score(ytest, y_pred)

0.8824

In [63]:
#Perceptron avec preprocessing
start_time = time.time()
pipe_pctr = Pipeline([('minmax', MinMaxScaler()), ('pctr', Perceptron(tol=1e-3, alpha=0.001, validation_fraction=0.05, random_state=0))])
pipe_pctr.fit(xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = pipe_pctr.predict(xtest)

Temps entraînement = 11.19940710067749s


In [64]:
accuracy_score(ytest, y_pred)

0.8951

In [65]:
#MLP 1 HL 300 sans preprocessing
start_time = time.time()
MLP1HL300 = MLPClassifier(hidden_layer_sizes=(300), random_state=0).fit(xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = MLP1HL300.predict(xtest)

Temps entraînement = 69.59555888175964s


In [ ]:
print(classification_report(ytest, y_pred))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99       980
           1       0.99      0.99      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.98      0.98      1010
           4       0.98      0.98      0.98       982
           5       0.99      0.98      0.99       892
           6       0.99      0.98      0.99       958
           7       0.98      0.98      0.98      1028
           8       0.97      0.98      0.97       974
           9       0.98      0.98      0.98      1009

    accuracy                           0.98     10000
   macro avg       0.98      0.98      0.98     10000
weighted avg       0.98      0.98      0.98     10000



In [ ]:
accuracy_score(ytest, y_pred)

0.9827

In [14]:
#MLP 1 HL 300 avec preprocessing

start_time = time.time()
std_MLP1HL300 = Pipeline([('minmax', MinMaxScaler()), ('mlp', MLPClassifier(hidden_layer_sizes=(300), random_state=0))]).fit(xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = std_MLP1HL300.predict(xtest)

Temps entraînement = 59.30945086479187s


In [69]:
accuracy_score(ytest, y_pred)

0.9653

In [106]:
start_time = time.time()
std_MLP2HL500300 = MLPClassifier(solver='adam', hidden_layer_sizes=(500, 300), tol=1e-3,batch_size=250, max_iter=100, alpha=0.1, random_state=0)
#std_MLP1HL300.out_activation_='softmax'
std_MLP2HL500300.fit(xtrain2, ytrain2)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = std_MLP2HL500300.predict(xtest)

Temps entraînement = 83.25282430648804s


In [107]:
accuracy_score(ytest, y_pred)

0.9681

In [108]:
std_MLP2HL500300.n_iter_

58

In [102]:
std_MLP2HL500300.out_activation_

'softmax'

In [1]:
start_time = time.time()
std_MLP2HL500300 = MLPClassifier(solver='adam', hidden_layer_sizes=(500, 300),
                                 tol=1e-4, batch_size=256, max_iter=50,
                                 alpha=0.01, random_state=0)
#std_MLP1HL300.out_activation_='softmax'
std_MLP2HL500300.fit(xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = std_MLP2HL500300.predict(xtest)

NameError: ignored

In [2]:
accuracy_score(ytest, y_pred)

NameError: ignored

In [153]:
std_MLP2HL500300.n_iter_

50

In [ ]:
start_time = time.time()
std_MLP2HL500300 = MLPClassifier(solver='adam', hidden_layer_sizes=(500, 300),
                                 tol=1e-4, batch_size=300, max_iter=150,
                                 alpha=0.01, random_state=0)
#std_MLP1HL300.out_activation_='softmax'
std_MLP1HL300.fit(pca_xtrain, ytrain)
end_time = time.time()
print(f"Temps entraînement = {end_time - start_time}s")
y_pred = std_MLP1HL300.predict(pca_xtest)

In [ ]:
accuracy_score(ytest, y_pred)

In [8]:
"""
param_grid = {
    'max_iter': [50, 100, 150],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.005, 0.05, 0.5, 1],
    'learning_rate': ['constant','adaptive'],
    'batch_size': [100, 200, 300]
}
"""


param_grid = {
    'max_iter': [50, 100, 150],
    'solver': ['sgd', 'adam', 'lbfgs'],
    'alpha': [0.005, 0.05, 0.5, 1],
}

In [13]:
grid = GridSearchCV(MLPClassifier(random_state=0), param_grid, cv=5)
grid.fit(xtrain2, ytrain2)

print(grid.best_params_) 

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (50) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilaye

{'alpha': 1, 'max_iter': 150, 'solver': 'lbfgs'}


/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


In [16]:
grid.best_params_

{'alpha': 1, 'max_iter': 150, 'solver': 'lbfgs'}